In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as pl
import matplotlib.pyplot as plt

import wradlib
from osgeo import osr
import datetime as dt
import glob
from osgeo import gdal
from scipy.spatial import cKDTree
import wradlib as wrl
import matplotlib as mpl
import math
%matplotlib inline 
#%matplotlib nbagg
#%matplotlib notebook
import datetime as dt

In [ ]:
## Load DWD Station Data
## ----------------------
dwd_time = np.load('/automount/ags/velibor/data/dwd_station/DWD_time.npy')
dwd_h = np.load('/automount/ags/velibor/data/dwd_station/DWD_h.npy')


dwd_x = np.load('/automount/ags/velibor/data/dwd_station/DWD_x.npy')
dwd_y = np.load('/automount/ags/velibor/data/dwd_station/DWD_y.npy')

dwd_pp = np.load('/automount/ags/velibor/data/dwd_station/DWD_pp.npy')
dwd_rf = np.load('/automount/ags/velibor/data/dwd_station/DWD_rf.npy')
dwd_tau = np.load('/automount/ags/velibor/data/dwd_station/DWD_tau.npy')

dwd_temp = np.load('/automount/ags/velibor/data/dwd_station/DWD_TEMP.NPY')

In [ ]:
pfad[0]

In [ ]:
%%time

## Load GPM DPR Data
## ----------------------

pfad = glob.glob("/automount/ags/velibor/gpmdata/dumpdataV7/dpr_ry_NS/dprrado_N*")
print(len(pfad))
pfad = sorted(pfad)
#for j in sorted(pfad):
for ii in range(len(pfad)):
    
    j = pfad[ii]
    print(j)
    
    sr_dwd_time = np.array([])
    sr_dwd_x = np.array([])
    sr_dwd_y = np.array([])
    sr_dwd_h = np.array([])
    sr_dwd_temp = np.array([])
    
    sr_dwd_rf = np.array([])
    sr_dwd_pp = np.array([])
    sr_dwd_tau = np.array([])
    
    sr_dwd_dist = np.array([])
    
    DR = h5py.File(j, 'r')

    sr_times = DR['SR']['sr_times'].value

    sr_time1 = DR['SR']['sr_time'].value
    sr_time = dt.datetime.strptime(sr_time1, "%Y.%m.%d -- %H:%M:%S")

    sr_x = DR['SR']['sr_x'].value
    sr_y = DR['SR']['sr_y'].value

    ## DPR Time on the same dtype as DWD time!!!!!!!!!

    sr_time = np.array(sr_time,dtype='datetime64[s]')
    dwd_time = np.array(dwd_time,dtype='datetime64[s]')

    #sr_bbh = DR['SR']['sr_bbh'].value
    #sr_typ = DR['SR']['sr_type'].value    
    #sr_typ = sr_typ//10000000

    #print(i)
    # Berechne delta t in sekunden integer
    delta_t = np.array(abs(sr_time - dwd_time), dtype='int')
    
    # bestimme INDEX fuer kleiner als 600 sekunden (10min)
    t_idx = np.where(delta_t<600)

    # neue array mit zeit index
    new_x = dwd_x[t_idx]
    new_y = dwd_y[t_idx]
    new_temp = dwd_temp[t_idx]
    new_h = dwd_h[t_idx]
    new_time = dwd_time[t_idx]
    new_rf = dwd_rf[t_idx]
    new_pp = dwd_pp[t_idx]
    new_tau = dwd_tau[t_idx]

    for i in range(len(sr_x)):
        # berechne distanzen
        dist = np.sqrt( (new_x - sr_x[i])**2 + (new_y - sr_y[i])**2 )

        #suche naheliegenste
        d_idx = np.abs(dist).argmin()

        res_h = new_h[d_idx]
        res_temp = new_temp[d_idx]
        res_x = new_x[d_idx]
        res_y = new_y[d_idx]
        res_time = new_time[d_idx] 
        res_rf = new_rf[d_idx]
        res_pp = new_pp[d_idx]
        res_tau = new_tau[d_idx]
        
        sr_dwd_x = np.append(sr_dwd_x, res_x)
        sr_dwd_y = np.append(sr_dwd_y, res_y)
        sr_dwd_h = np.append(sr_dwd_h, res_h)
        sr_dwd_temp = np.append(sr_dwd_temp, res_temp)
        sr_dwd_rf = np.append(sr_dwd_rf, res_rf)
        sr_dwd_pp = np.append(sr_dwd_pp, res_pp)
        sr_dwd_tau = np.append(sr_dwd_tau, res_tau)
        sr_dwd_dist = np.append(sr_dwd_dist, dist[d_idx])
        #sr_dwd_time = np.append(sr_dwd_time, res_time)
        
        
    np.save('/automount/ags/velibor/data/dwd_station_gpm/dwd_on_dpr_x_'+str(sr_time1)+'.npy', sr_dwd_x)
    np.save('/automount/ags/velibor/data/dwd_station_gpm/dwd_on_dpr_y_'+str(sr_time1)+'.npy', sr_dwd_y)
    np.save('/automount/ags/velibor/data/dwd_station_gpm/dwd_on_dpr_h_'+str(sr_time1)+'.npy', sr_dwd_h)
    np.save('/automount/ags/velibor/data/dwd_station_gpm/dwd_on_dpr_temp_'+str(sr_time1)+'.npy', sr_dwd_temp)
    np.save('/automount/ags/velibor/data/dwd_station_gpm/dwd_on_dpr_rf_'+str(sr_time1)+'.npy', sr_dwd_rf)
    np.save('/automount/ags/velibor/data/dwd_station_gpm/dwd_on_dpr_p_'+str(sr_time1)+'.npy', sr_dwd_pp)
    np.save('/automount/ags/velibor/data/dwd_station_gpm/dwd_on_dpr_tau_'+str(sr_time1)+'.npy', sr_dwd_tau)
    np.save('/automount/ags/velibor/data/dwd_station_gpm/dwd_on_dpr_dist_'+str(sr_time1)+'.npy', sr_dwd_dist)

In [ ]:
"""np.save('/automount/ftp/velibor/data/dwd_on_dpr/dwdondpr_x.npy', sr_dwd_x)
np.save('/automount/ftp/velibor/data/dwd_on_dpr/dwdondpr_y.npy', sr_dwd_y)
np.save('/automount/ftp/velibor/data/dwd_on_dpr/dwdondpr_z.npy', sr_dwd_h)
np.save('/automount/ftp/velibor/data/dwd_on_dpr/dwdondpr_temp.npy', sr_dwd_temp)
np.save('/automount/ftp/velibor/data/dwd_on_dpr/dwdondpr_rf.npy', sr_dwd_rf)
np.save('/automount/ftp/velibor/data/dwd_on_dpr/dwdondpr_pres.npy', sr_dwd_pp)
np.save('/automount/ftp/velibor/data/dwd_on_dpr/dwdondpr_tau.npy', sr_dwd_tau)
np.save('/automount/ftp/velibor/data/dwd_on_dpr/dwdondpr_dist.npy', sr_dwd_dist)
"""

In [ ]:
import glob


dname = {0:'x', 1:'y', 2:'h', 3:'temp', 4:'rf', 5:'p', 6:'tau', 7:'dist'}

for jj in range(len(dname)):
    print (jj)

    pfad = glob.glob('/automount/ags/velibor/data/dwd_station_gpm/*_' + dname[jj] + '*.npy')
    pfad = sorted(pfad)

    dwd_on_dpr = np.array([])

    for i in pfad:
        data = np.load(i)
        #print(i)
        dwd_on_dpr = np.append(dwd_on_dpr, data)

    np.save('/automount/ftp/velibor/data/dwd_on_dpr/dwdondpr_all_' + dname[jj] + '.npy', dwd_on_dpr)    

In [ ]:
sr_dwd_rf[sr_dwd_rf==-999.]=np.nan
plt.scatter(sr_dwd_x, sr_dwd_y, c=sr_dwd_dist)
plt.colorbar()
plt.xlim(-600,400)
plt.ylim(-4500, -3700)
plt.grid()

In [ ]:
%%time
plt.scatter(sr_x,sr_y)
plt.xlim(-600,400)
plt.ylim(-4500, -3700)
plt.grid()